In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [53]:
matches = pd.read_csv("teams_pl_23-24_fbref.csv")
stadiums = pd.read_csv("stadiums.csv")

In [54]:
matches.rename(columns={"venue_date": "date", "venue_time": "time"}, inplace=True)
matches = matches.drop(columns = ["home_shirtnumber", "home_nationality", "home_position", "home_age", "away_shirtnumber", "away_nationality", "away_position", "away_age"])

In [55]:
matches["home_goals"] = matches["home_goals"] + matches["away_own_goals"]
matches["away_goals"] = matches["away_goals"] + matches["home_own_goals"]

#### Add stadium info to every match

In [56]:
matches_with_stadiums = pd.merge(matches, stadiums[stadiums["Closed"].isna()], left_on=['home_team'], right_on=['Club'], how='left').drop(columns=["Closed", "Opened"])
matches_with_stadiums.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude
0,2023-2024,2023-08-11,20:00,1,21572,Craig Pawson,5-4-1,

#### Add columns with match outcome and points of each team

In [57]:
def determine_outcome_and_points(row):
    if row['home_goals'] > row['away_goals']:
        return 1, 3, 0  # Home win
    elif row['home_goals'] < row['away_goals']:
        return 2, 0, 3  # Away win
    else:
        return 0, 1, 1  # Draw  
    
matches_with_stadiums[['outcome', 'home_points', 'away_points']] = matches_with_stadiums.apply(lambda row: pd.Series(determine_outcome_and_points(row)), axis=1)

In [58]:
pd.set_option('display.max_columns', None)
matches_with_stadiums.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude,outcome,home_points,away_points
0,2023-2024,2023-08-11,2

In [59]:
cols = matches_with_stadiums.columns
attributes = []
for col in cols:
    if col.startswith("home_"):
        attributes.append(col.replace("home_", ""))
attributes = attributes[1:] # delete team attribute

In [60]:
standings_columns = ['matches_played', 'wins', 'draws', 'defeats', 'goal_difference', 'goals_conceded'] + attributes
standings = pd.DataFrame(columns=['season', 'team'] + standings_columns)
matches_sorted = matches_with_stadiums.sort_values(by=['season', 'date'])

In [61]:
for season in matches_sorted['season'].unique():
    # Filter data for the current season
    season_data = matches_sorted[matches_sorted['season'] == season]
    teams = season_data['home_team'].unique()
    
    standings_dict = {team: {attribute: 0 for attribute in standings_columns} for team in teams}

    # tu można dodać punkty karne, bardzo niewiele to zmieni ale jednak w teorii dokładniejszy model!!!!!!
    
    for dt in season_data['date'].unique():
        # Filter data for the current date
        round_data = season_data[season_data['date'] == dt]
        
        # Update standings for each match on current date
        for index, row in round_data.iterrows():
            home_team = row['home_team']
            away_team = row['away_team']
            home_goals = row['home_goals']
            away_goals = row['away_goals']
            home_points = row['home_points']
            away_points = row['away_points']
            outcome = row['outcome']
            
            # Update home team standings
            standings_dict[home_team]['matches_played'] += 1
            standings_dict[home_team]['goals_conceded'] += away_goals
            
            for attr in attributes:
                standings_dict[home_team][attr] += row["home_" + attr]

            standings_dict[home_team]['goal_difference'] = standings_dict[home_team]['goals'] - standings_dict[home_team]['goals_conceded']
            
            # Update away team standings
            standings_dict[away_team]['matches_played'] += 1
            standings_dict[away_team]['goals_conceded'] += home_goals
            
            for attr in attributes:
                standings_dict[away_team][attr] += row["away_" + attr]

            standings_dict[away_team]['goal_difference'] = standings_dict[away_team]['goals'] - standings_dict[away_team]['goals_conceded']

            # Update W L D
            if outcome == 1:
                standings_dict[home_team]['wins'] += 1
                standings_dict[away_team]['defeats'] += 1
            elif outcome == 0:
                standings_dict[home_team]['draws'] += 1
                standings_dict[away_team]['draws'] += 1
            else:
                standings_dict[home_team]['defeats'] += 1
                standings_dict[away_team]['wins'] += 1

        temp_standings_data = {
        'season': season,
        'date': dt,
        'team': list(standings_dict.keys()),
        'matches_played': [standings_dict[team]['matches_played'] for team in standings_dict],
        'goals_conceded': [standings_dict[team]['goals_conceded'] for team in standings_dict],
        'goal_difference': [standings_dict[team]['goal_difference'] for team in standings_dict],
        'wins': [standings_dict[team]['wins'] for team in standings_dict],
        'draws': [standings_dict[team]['draws'] for team in standings_dict],
        'defeats': [standings_dict[team]['defeats'] for team in standings_dict]}

        for attr in attributes:
            temp_standings_data[attr] = [standings_dict[team][attr] for team in standings_dict]

        temp_standings = pd.DataFrame(temp_standings_data)

        standings = pd.concat([standings, temp_standings])
standings = standings.sort_values(by=['season', 'date', 'points', 'goal_difference', 'goals'], ascending=[True, True, False, False, False])
standings.reset_index(drop=True, inplace=True)

C:\Users\MateuszAndryszak\AppData\Local\Temp\ipykernel_7036\4145182772.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  standings = pd.concat([standings, temp_standings])


In [62]:
standings[standings["matches_played"] == 38].head(2)

,season,team,matches_played,wins,draws,defeats,goal_difference,goals_conceded,minutes,goals,assists,pens_made,pens_att,shots,shots_on_target,cards_yellow,cards_red,touches,tackles,interceptions,blocks,xg,npxg,xg_assist,sca,gca,passes_completed,passes,passes_pct,progressive_passes,carries,progressive_carries,take_ons,take_ons_won,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,passes_medium,passes_pct_medium,passes_completed_long,passes_long,passes_pct_long,pass_xa,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,passes_live,passes_dead,passes_free_kicks,through_balls,passes_switches,crosses,throw_ins,corner_kicks,corner_kicks_in,corner_kicks_out,corner_kicks_straight,passes_offsides,passes_blocked,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,challenge_tackles,challenges,challenge_tackles_pct,challenges_lost,blocked_shots,blocked_passes,tackles_interceptions,clearances,errors,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,take_ons_won_pct,take_ons_tackled,take_ons_tackled_pct,carries_distance,carries_progressive_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,progressive_passes_received,cards_yellow_red,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,points,date
2380,2023-2024,Manchester City,38,28,7,3,62,34,37574,96,69,9,10,683,264,53,2,31414,517,233,353,82.1,74.5,60.9,1259,169,24616,27758,3359.9,2060,20210,1127,731,377,394064,112270,12895,13721,3563.1,9449,10306,3477.1,1652,2498,2500.0,62.1,545,1890,484,65,26136,1580,460,85,133,700,604,286,114,66,7,42,383,305,183,221,113,222,458,1793.2,236,75,278,750,457,17,2162,7012,14348,10260,1466,31404,1913.1,282,1509.5,105397,58071,867,415,441,356,24343,2032,1,289,410,42,8,3,1,1734,325,288,1998.0,91,2024-05-19
2381,2023-2024,Arsenal,38,28,5,5,62,29,37588,91,62,10,10,647,209,64,2,26493,610,292,412,78.4,70.4,56.3,1185,152,19094,22558,3188.9,2106,16297,821,671,283,309533,102210,9719,10643,3451.4,7809,8765,3358.5,1178,2162,2059.0,58.0,513,1546,548,68,20941,1530,443,104,65,670,560,265,217,1,0,87,319,348,257,246,107,271,543,1995.0,272,106,306,902,515,18,1974,6714,11407,8560,1480,26483,1620.5,304,1718.5,81171,41510,575,327,525,375,18917,2088,1,391,394,87,9,3,1,1752,503,499,1925.0,89,2024-05-19


In [63]:
matches_filtered = matches_with_stadiums[matches_with_stadiums['round'] > 3]
df = matches_filtered
df.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude,outcome,home_points,away_points
29,2023-2024,2023-09-01,

In [94]:
# rolling_features = [
#     'total_points', 'goals_scored', 'goals_conceded', 'goal_difference', 'Goal Attempts', 'Shots on Goal', 'Shots off Goal',
#     'Blocked Shots', 'Free Kicks', 'Corner Kicks', 'Offsides', 'Throw-ins',
#     'Goalkeeper Saves', 'Fouls', 'Yellow Cards', 'Total Passes', 'Completed Passes',
#     'Tackles', 'Attacks', 'Dangerous Attacks', 'Red Cards', 'Expected Goals (xG)',
#     'Clearances Completed'
#     ]
# for index, row in df.iterrows():
#     date = row["date"]
#     season = row["season"]
#     home_team = row["home_team"]
#     away_team = row["away_team"]

#     home_team_matches_played = standings[(standings["date"] == row["date"]) & (standings["team"] == home_team)]["matches_played"].values[0] - 1
#     away_team_matches_played = standings[(standings["date"] == row["date"]) & (standings["team"] == away_team)]["matches_played"].values[0] - 1

#     home_team_stats = standings[(standings["season"] == season) & (standings["team"] == home_team) & (standings["matches_played"] == home_team_matches_played)].iloc[0]
#     away_team_stats = standings[(standings["season"] == season) & (standings["team"] == away_team) & (standings["matches_played"] == away_team_matches_played)].iloc[0]

#     home_team_matches_played_old = max(0, home_team_matches_played - 5)
#     away_team_matches_played_old = max(0, away_team_matches_played - 5)

#     home_team_matches_played_actual = min(5, home_team_matches_played)
#     away_team_matches_played_actual = min(5, away_team_matches_played)

#     if home_team_matches_played_actual > 5:
#         home_team_stats_old = standings[(standings["season"] == season) & (standings["team"] == home_team) & (standings["matches_played"] == home_team_matches_played_old)].iloc[0]
#     if away_team_matches_played_actual > 5:
#         away_team_stats_old = standings[(standings["season"] == season) & (standings["team"] == away_team) & (standings["matches_played"] == away_team_matches_played_old)].iloc[0]


#     for feature in rolling_features:
#         if home_team_matches_played_actual <= 5:
#             row["home_last5_" + feature] = home_team_stats[feature]
#         else:
#             row["home_last5_" + feature] = home_team_stats[feature] - home_team_stats_old[feature]

#         if away_team_matches_played_actual <= 5:
#             row["away_last5_" + feature] = away_team_stats[feature]
#         else:  
#             row["away_last5_" + feature] = away_team_stats[feature] - away_team_stats_old[feature]

# df    

In [64]:
# Function to get team stats
def get_team_stats(season, team, matches_played):
    return standings[(standings["season"] == season) & (standings["team"] == team) & (standings["matches_played"] == matches_played)].iloc[0]

# Function to calculate rolling stats
def calculate_rolling_stats(stats, stats_old, matches_played, real_matches_played):
    rolling_stats = {}
    for feature in standings_columns:
        if matches_played == 0:
            rolling_stats[feature] = stats[feature]
            if real_matches_played != 0:
                rolling_stats[feature] = rolling_stats[feature] / real_matches_played
        else:
            rolling_stats[feature] = stats[feature] - stats_old[feature]
            rolling_stats[feature] = rolling_stats[feature] / 5
    return rolling_stats

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    date = row["date"]
    season = row["season"]
    home_team = row["home_team"]
    away_team = row["away_team"]

    home_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == home_team)]["matches_played"].values[0] - 1
    away_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == away_team)]["matches_played"].values[0] - 1

    home_team_stats = get_team_stats(season, home_team, home_team_matches_played)
    away_team_stats = get_team_stats(season, away_team, away_team_matches_played)

    home_team_matches_played_old = max(0, home_team_matches_played - 5)
    away_team_matches_played_old = max(0, away_team_matches_played - 5)

    if home_team_matches_played_old > 0:
        home_team_stats_old = get_team_stats(season, home_team, home_team_matches_played_old)
    if away_team_matches_played_old > 0:
        away_team_stats_old = get_team_stats(season, away_team, away_team_matches_played_old)

    home_rolling_stats = calculate_rolling_stats(home_team_stats, home_team_stats_old if home_team_matches_played_old > 0 else None, home_team_matches_played_old, home_team_matches_played)
    away_rolling_stats = calculate_rolling_stats(away_team_stats, away_team_stats_old if away_team_matches_played_old > 0 else None, away_team_matches_played_old, away_team_matches_played)

    for feature, value in home_rolling_stats.items():
        df.at[index, f"home_last5_{feature}"] = value
    for feature, value in away_rolling_stats.items():
        df.at[index, f"away_last5_{feature}"] = value

    for feature in home_team_stats.index:
        if feature in standings_columns:
            df.at[index, f"home_{feature}"] = home_team_stats[feature]
            df.at[index, f"away_{feature}"] = away_team_stats[feature]
            if home_team_matches_played > 0:
                df.at[index, f"home_{feature}"] = home_team_stats[feature] / home_team_matches_played
            if away_team_matches_played > 0:
                df.at[index, f"away_{feature}"] = away_team_stats[feature] / away_team_matches_played


C:\Users\MateuszAndryszak\AppData\Local\Temp\ipykernel_7036\2763846368.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f"home_last5_{feature}"] = value
C:\Users\MateuszAndryszak\AppData\Local\Temp\ipykernel_7036\2763846368.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f"home_last5_{feature}"] = value
C:\Users\MateuszAndryszak\AppData\Local\Temp\ipykernel_7036\2763846368.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [66]:
df.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude,outcome,home_points,away_points,home_last5_matches_playe